# Get Songs from Spotify Playlist

In [79]:
# Libraries
import requests
import re
import json
import config

In [80]:
# Get token
url = "https://accounts.spotify.com/api/token"

payload=f'client_id={config.CLIENT_ID}&client_secret={config.CLIENT_SECRET}&grant_type=client_credentials'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': '__Host-device_id=AQA1E_8u1HHZzuE7O9d_fV4YwanfNozg6q_mdfweHpowhpZGWG9saI45tX6W5no4APxdKSdgb0ZqexZNfHeYiXVZhLhlP5JYj0M; sp_tr=false'
}

response = requests.request("POST", url, headers=headers, data=payload)

AccessToken = json.loads(response.text)['access_token']
AccessToken

'BQAYuNNhwXJ0B6jxsopoOsemNCmIc8tQ4_1O5AA6MGHRNbCjvsH_rE8wnlpU4UZ5u83kgKgI9hOwTKt1EsPsBC5_o1hgE9rXP2tVUwjs2yUrWGB4FDDp'

## Access Token
Access token is required to call Spotify API. In this case, the Access token needs to be refreshed every time the request is called. The Spotify credential used is the creator of the code.

In [81]:
# Sample Link
link = 'https://open.spotify.com/playlist/3EZXILd3Au7L8Wna75GUz3?si=cc22f026f7e34fd7'

In [82]:
# Get Link ID
import re
playlist_id = re.search(r'(?<=playlist/)\w+', link).group()

playlist_id

'3EZXILd3Au7L8Wna75GUz3'

In [83]:
import requests

url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

payload={}
headers = {
  'Authorization': f'Bearer {AccessToken}'
}

file = open('response.json', 'w')
response = requests.request("GET", url, headers=headers, data=payload)

file.write(response.text)
file.close()

In [85]:
import json

file = open('response.json', 'r')

# Load the contents of the file into a Python object
data = json.load(file)

# Close the file object
file.close()

# Get all song ID inside the playlist
id_list = []

id_list = [song['track']['id'] for song in data['items']]

id_list

['0xcSj8Zchf6OwaSOAPAkbg', '1nTzXNZCnZ0T7S8BqkL98r']

In [62]:
def getSongDetails(song_id):
    url = f"https://api.spotify.com/v1/tracks/{song_id}"

    payload={}
    headers = {
        'Authorization': f'Bearer {AccessToken}'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    print(response.text)

In [86]:
getSongDetails('0xcSj8Zchf6OwaSOAPAkbg')

{
  "album" : {
    "album_group" : "single",
    "album_type" : "single",
    "artists" : [ {
      "external_urls" : {
        "spotify" : "https://open.spotify.com/artist/1XlLhtgJjC4ROQZilBZAvw"
      },
      "href" : "https://api.spotify.com/v1/artists/1XlLhtgJjC4ROQZilBZAvw",
      "id" : "1XlLhtgJjC4ROQZilBZAvw",
      "name" : "Nic D",
      "type" : "artist",
      "uri" : "spotify:artist:1XlLhtgJjC4ROQZilBZAvw"
    }, {
      "external_urls" : {
        "spotify" : "https://open.spotify.com/artist/5zixe6AbgXPqt4c1uSl94L"
      },
      "href" : "https://api.spotify.com/v1/artists/5zixe6AbgXPqt4c1uSl94L",
      "id" : "5zixe6AbgXPqt4c1uSl94L",
      "name" : "Connor Price",
      "type" : "artist",
      "uri" : "spotify:artist:5zixe6AbgXPqt4c1uSl94L"
    } ],
    "available_markets" : [ "AD", "AE", "AG", "AL", "AM", "AO", "AR", "AT", "AU", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BN", "BO", "BR", "BS", "BT", "BW", "BY", "BZ", "CA", "CD", "CG", "CH", "CI", 